# Fill Location
---
user 데이터의 변수들 중, location 변수의 null 값을 처리하는 코드이다.

아래 알고리즘과 같이 값을 채우려 한다.

```python
for user in user_list:
    if country is null:
        ref (same city user)

    if state is null:
        ref (same country & city user)
```

In [2]:
import pandas as pd
import numpy as np

In [3]:
users = pd.read_csv('data/users_f_location.csv')
ratings_train = pd.read_csv('data/train_ratings.csv')
ratings_test = pd.read_csv('data/test_ratings.csv')
ratings = pd.concat([ratings_train, ratings_test], axis=0)

del ratings_test, ratings_train

# location이 모두 null인 유저
---

In [4]:
users[users.location_city.isna() &
      users.location_country.isna() &
      users.location_state.isna()]
      

,user_id,age,location_city,location_state,location_country
2,11676,NaN,NaN,NaN,NaN
1679,146804,NaN,NaN,NaN,NaN
7760,70063,NaN,NaN,NaN,NaN
8634,112345,39.0,NaN,NaN,NaN
38772,156948,NaN,NaN,NaN,NaN
48206,169489,NaN,NaN,NaN,NaN
58435,56072,NaN,NaN,NaN,NaN
60682,64582,30.0,NaN,NaN,NaN
67104,218092,NaN,NaN,NaN,NaN


In [5]:
null_user = users[users.location_city.isna() &
      users.location_country.isna() &
      users.location_state.isna()].user_id.values

ratings.query('user_id in @null_user').user_id.value_counts()
      

11676     6939
146804       7
70063        5
156948       1
56072        1
64582        1
218092       1
112345       1
169489       1
Name: user_id, dtype: int64

location 정보가 모두 null인 유저 중, user_id가 11676인 사람은 rating 기록이 6939개 가량 있음.


# location 정보 채우기
---
pseudo algorithm:


### 모두 null인 유저 제거

In [6]:
# 제거하기전 저장
all_null = users[users.location_city.isna() &
      users.location_country.isna() &
      users.location_state.isna()]

In [7]:
# 모두 null인 유저 제거
users = users.drop(all_null.index, axis=0)

## country가 null인 경우 채우기

In [8]:
country_null_v = users[users.location_country.isna() & 
      users.location_state.notna() &
      users.location_city.notna()
].location_city.values

print(len(country_null_v))

93


In [9]:
for city in country_null_v:
    if len(users.query('location_city == @city')) == 1:
        # null 값인 데이터가 유일한 경우(참조할 수 있는 다른 값이 없음)
        continue
    
    mode_value = users.query('location_city == @city').location_country.mode()
    if len(mode_value.values) == 0:
        # 유일하진 않지만 null 값인 데이터만 있는 경우(참조할 수 있는 다른 값이 없음)
        continue
    null_df_idx = users.query('location_city == @city').location_country.isna().index

    for idx in null_df_idx:
        users.loc[idx, 'location_country'] = mode_value.values[0]

In [10]:
country_null_v = users[users.location_country.isna() & 
      users.location_state.notna() &
      users.location_city.notna()
].location_city.values

len(country_null_v)

53

null 값 개수 93 -> 53

## state 채우기

In [11]:
state_null_v = users[users.location_country.notna() & 
      users.location_state.isna() &
      users.location_city.notna()
].loc[:, ['location_city', 'location_country']].values

print(len(state_null_v))

926


In [12]:
for city, country in state_null_v:
    if len(users.query('location_city == @city and location_country == @country')) == 1:
        # null 값인 데이터가 유일한 경우(참조할 수 있는 다른 값이 없음)
        continue
    
    mode_value = users.query('location_city == @city and location_country == @country').location_state.mode()
    if len(mode_value.values) == 0:
        # 유일하진 않지만 null 값인 데이터만 있는 경우(참조할 수 있는 다른 값이 없음)
        continue
    
    null_df_idx = users.query('location_city == @city and location_country == @country').location_state.isna().index

    for idx in null_df_idx:
        users.loc[idx, 'location_state'] = mode_value.values[0]

In [13]:
state_null_v = users[users.location_country.notna() & 
      users.location_state.isna() &
      users.location_city.notna()
].loc[:, ['location_city', 'location_country']].values

print(len(state_null_v))

352


null 값 개수 962 -> 352

## 최종 null value 정리

In [14]:
# 기존에 빼뒀던 location이 모두 null인 user 다시 Concat
users = pd.concat([users, all_null], axis=0)

In [15]:
users[users.location_state.isna() |
      users.location_city.isna() |
      users.location_country.isna()]

,user_id,age,location_city,location_state,location_country
633,271622,NaN,malmok,NaN,aruba
1008,48630,34.0,NaN,newjersey,usa
1012,49460,37.0,NaN,kansas,usa
1065,56399,63.0,NaN,surrey,unitedkingdom
1086,59756,25.0,belgrade,NaN,yugoslavia
...,...,...,...,...,...
38772,156948,NaN,NaN,NaN,NaN
48206,169489,NaN,NaN,NaN,NaN
58435,56072,NaN,NaN,NaN,NaN
60682,64582,30.0,NaN,NaN,NaN


In [16]:
(len(users[users.location_state.isna() |
      users.location_city.isna() |
      users.location_country.isna()]
) / len(users))

0.009957116841919756

location에서 하나라도 비어있는 값의 개수는 678, 약 1%

In [17]:
users.to_csv('users_f_location_1.1.csv', index=None)